In [199]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
import time
import urllib3
import json

In [200]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    urllib3.disable_warnings()
    parser = html.fromstring(resp)


    return parser

In [201]:
def links_proyectos(parser):

    links1 = parser.xpath('//div[@class="block5_widget_content row-2-col"]//a/@href')
    
    urls1 = []
    for link in links1:
        if link.startswith('https://'):
            urls1.append(link.strip())

    urls1 = list(set(urls1))

    links_bottons= parser.xpath('//div[@class="wpsm_panel-group"]//a/@href')

    urls2 = []
    for link in links_bottons:
        if link.startswith('https://'):
            urls2.append(link.strip())

    urls2 = list(set(urls2))

    urls1.extend(urls2)

    return urls1

In [202]:
## Parser semilla y Links Proyectos
parsert = parser('https://www.clacso.org/convocatorias/')
links_proy = links_proyectos(parsert)

In [203]:
### Titulo

def titulo(parser_proy):
    
    titulo = parser_proy.xpath('//header/h1[@class="entry-title"]/text()')

    return titulo[0]

proy = links_proy[0]
parser_proy = parser(proy)
tituloT = titulo(parser_proy)
tituloT

'Mes de la mujer: 100 becas de formación para mujeres y disidencias'

In [204]:
## Descripción

def descripcion(parser_proy):
    descripcion = parser_proy.xpath('//div[@class="pf-content"]//p/text()')

    desc_completa = ''
    for desc in descripcion:
        desc = desc.strip()
        if len(desc) != 0: 
            desc_completa = desc_completa + desc + ' '

    return desc_completa.strip()


proy = links_proy[40]
parser_proy = parser(proy)
descripcionT = descripcion(parser_proy)

descripcionT

'a) Realizar un artículo analítico que ponga en diálogo los temas trabajados en el seminario con los de los proyectos de investigación de cada estudiante. b) Seleccionar un foro del curso, sintetizando y problematizando los intercambios a partir de las preguntas y la bibliografía trabajadas en el curso. Presentar este análisis en forma de ensayo. c) Elaborar un trabajo monográfico que integre los conceptos o categorías tratadas en el seminario, específicamente. Los y deberán tener una extensión mínima de 10 y una extensión máxima de 15 carillas. El propósito de este video es introducir a los/as interesados/as al tema del seminario. En el siguiente link encontrarán algunos modelos de referencia que podrán ayudarles a realizar el suyo: Consultas: La convocatoria estuvo orientada a la selección de seminarios virtuales cuyos equipos docentes están integrados por investigadores e investigadoras de Redes de Posgrados, Grupos de Trabajo, y profesoras y profesores pertenecientes a Centros de C

In [217]:
### Ano convocatoria

def ano(parser_proy):
    ano = parser_proy.xpath('//script[@class="yoast-schema-graph"]/text()')
    diccionario = json.loads(ano[0])
    ano = diccionario['@graph'][2]['datePublished'].split('-')[0]

    return ano

proy = links_proy[45]

parser_proy = parser(proy)

anoT = ano(parser_proy)

anoT

'2019'

In [206]:
clacso = pd.DataFrame()

clacso['Titulo'] = tituloT
clacso['Descripción'] = desc_completa
clacso['Ano'] = anoT

,Titulo,Descripción
